In [24]:
import pandas as pd
import pprint

In [45]:
pp = pprint.PrettyPrinter(indent=2)

input_file = r'.\gg_fare_filing.xlsx'
orig = 'NYC'
currency = 'USD'

In [35]:
df_input = pd.read_excel(input_file, sheet_name='input', na_filter=False)
df_input.head()

,dest,booking_class,season,base_fare,direct
0,TPE,Y,L,2300,
1,TPE,B,L,2265,
2,TPE,C,L,5000,D
3,SGN,M,L,1000,
4,SGN,Q,L,800,


In [36]:
df_cabin_mapping = pd.read_excel(input_file, sheet_name='cabin_mapping')
df_cabin_mapping.head()

,booking_class,cabin
0,W,Premium Economy
1,U,Premium Economy
2,A,Premium Economy
3,E,Premium Economy
4,Y,Economy


In [37]:
df_season_mapping = pd.read_excel(input_file, sheet_name='season_mapping')
df_season_mapping.head()

,season,season_code
0,L,L
1,K1,K
2,K2,K
3,H1,H
4,H2,H


In [38]:
df_input = pd.merge(df_input, df_cabin_mapping, on='booking_class')
df_input = pd.merge(df_input, df_season_mapping, on='season')
df_input.head()

,dest,booking_class,season,base_fare,direct,cabin,season_code
0,TPE,Y,L,2300,,Economy,L
1,TPE,B,L,2265,,Economy,L
2,TPE,C,L,5000,D,Business,L
3,SGN,M,L,1000,,Economy,L
4,SGN,Q,L,800,,Economy,L


In [39]:
df_fare_combination = pd.read_excel(input_file, sheet_name='fare_combination', na_filter=False)
df_fare_combination.head()

,weekend,weekend_surcharge,oneway,oneway_multiplier,oneway_mapping
0,X,0,,1.0,RT
1,W,80,,1.0,RT
2,X,0,O,0.6,OO
3,W,80,O,0.6,OO


In [40]:
def gen_fares():
    output = []

    # loop through each input row
    for i, row_input in df_input.iterrows():
        dest = row_input['dest']
        booking_class = row_input['booking_class']
        season = row_input['season']
        season_code = row_input['season_code']
        base_fare = row_input['base_fare']
        direct = row_input['direct']
        cabin = row_input['cabin']
        
        # create different weekend and oneway combinations
        for j, row_fare_combination in df_fare_combination.iterrows():
            weekend = row_fare_combination['weekend']
            oneway = row_fare_combination['oneway']
            weekend_surcharge = row_fare_combination['weekend_surcharge']
            oneway_multiplier = row_fare_combination['oneway_multiplier']
            oneway_mapping = row_fare_combination['oneway_mapping']

            fare_basis = booking_class + season_code + weekend + 'S' + oneway + direct + 'E'
            fare = (base_fare + weekend_surcharge) * oneway_multiplier
            
            row_output = {
                'orig': orig,
                'dest': dest,
                'fare_basis': fare_basis,
                'booking_class': booking_class,
                'cabin': cabin,
                'ow/rt': oneway_mapping,
                'blank1': '',
                'blank2': '',
                'blank3': '',
                'currency': currency,
                'fare': fare
            }
            
            output.append(row_output)
    return output

In [41]:
pp.pprint(gen_fares())

[ { 'blank1': '',
    'blank2': '',
    'blank3': '',
    'booking_class': 'Y',
    'cabin': 'Economy',
    'currency': 'USD',
    'dest': 'TPE',
    'fare': 2300.0,
    'fare_basis': 'YLXSE',
    'orig': 'NYC',
    'ow/rt': 'RT'},
  { 'blank1': '',
    'blank2': '',
    'blank3': '',
    'booking_class': 'Y',
    'cabin': 'Economy',
    'currency': 'USD',
    'dest': 'TPE',
    'fare': 2380.0,
    'fare_basis': 'YLWSE',
    'orig': 'NYC',
    'ow/rt': 'RT'},
  { 'blank1': '',
    'blank2': '',
    'blank3': '',
    'booking_class': 'Y',
    'cabin': 'Economy',
    'currency': 'USD',
    'dest': 'TPE',
    'fare': 1380.0,
    'fare_basis': 'YLXSOE',
    'orig': 'NYC',
    'ow/rt': 'OO'},
  { 'blank1': '',
    'blank2': '',
    'blank3': '',
    'booking_class': 'Y',
    'cabin': 'Economy',
    'currency': 'USD',
    'dest': 'TPE',
    'fare': 1428.0,
    'fare_basis': 'YLWSOE',
    'orig': 'NYC',
    'ow/rt': 'OO'},
  { 'blank1': '',
    'blank2': '',
    'blank3': '',
    'booking_cla

In [42]:
output = gen_fares()

In [43]:
columns = [
    'orig', 
    'dest', 
    'fare_basis', 
    'booking_class', 
    'cabin', 
    'ow/rt', 
    'blank1', 
    'blank2', 
    'blank3', 
    'currency', 
    'fare'
]
df_output = pd.DataFrame(columns=columns, data=output)
df_output

,orig,dest,fare_basis,booking_class,cabin,ow/rt,blank1,blank2,blank3,currency,fare
0,NYC,TPE,YLXSE,Y,Economy,RT,,,,USD,2300.0
1,NYC,TPE,YLWSE,Y,Economy,RT,,,,USD,2380.0
2,NYC,TPE,YLXSOE,Y,Economy,OO,,,,USD,1380.0
3,NYC,TPE,YLWSOE,Y,Economy,OO,,,,USD,1428.0
4,NYC,TPE,BLXSE,B,Economy,RT,,,,USD,2265.0
5,NYC,TPE,BLWSE,B,Economy,RT,,,,USD,2345.0
6,NYC,TPE,BLXSOE,B,Economy,OO,,,,USD,1359.0
7,NYC,TPE,BLWSOE,B,Economy,OO,,,,USD,1407.0
8,NYC,TPE,CLXSDE,C,Business,RT,,,,USD,5000.0
9,NYC,TPE,CLWSDE,C,Business,RT,,,,USD,5080.0


In [46]:
sheet_name = 'copy_this'
with pd.ExcelWriter(input_file, engine="openpyxl", mode='a') as writer:
    workbook = writer.book
    try:
        workbook.remove(workbook[sheet_name])
    except:
        print('Worksheet does not exist')
    finally:
        df_output.to_excel(writer, sheet_name=sheet_name, index=False)